In [1]:
from tensorflow.keras.datasets import mnist
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
tf.compat.v1.disable_eager_execution()

In [2]:
path = os.getcwd() + "\mnist.npz"
(x_train, y_train), (x_test, y_test) = mnist.load_data(path=path)

print(f'x_train:{x_train.shape}')
print(f'y_train:{y_train.shape}')
print(f'x_test:{x_test.shape}')
print(f'y_test:{y_test.shape}')

x_train:(60000, 28, 28)
y_train:(60000,)
x_test:(10000, 28, 28)
y_test:(10000,)


In [3]:
n_inputs = 784
n_hidden_layer = 256
n_classes = 10
learn_rate = 0.001
batch_size = 128
training_epochs = 20
display_step = int(x_train.shape[0] / batch_size)


In [11]:
# #flatten input
# x_train = x_train.astype(np.float32)
# x_test = x_test.astype(np.float32)
# x_train = tf.reshape(x_train, [-1, n_inputs])
# x_test = tf.reshape(x_test, [-1, n_inputs])



# #normilize image
# x_train, x_test = x_train / 255.0, y_train / 255.0

# train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

# #one hot encoding outputs
# y_train = to_categorical(y_train.astype(np.float32))
# y_test = to_categorical(y_test.astype(np.float32))
(train_feature, train_label), (test_feature, test_label) = mnist.load_data(path=path)

train_feature, test_feature = np.array(train_feature, np.float32), np.array(test_feature, np.float32)
train_feature = train_feature.reshape(-1, train_feature.shape[0]).T
test_feature = test_feature.reshape(-1, test_feature.shape[0]).T

train_feature = train_feature / 255.0
test_feature = test_feature / 255.0
batch_size=32



train_label = to_categorical(train_label.astype(np.float32))
test_label = to_categorical(test_label.astype(np.float32))




In [18]:
print(f'x_train:{train_feature.shape}')
print(f'y_train:{train_label.shape}')
print(f'x_test:{test_feature.shape}')
print(f'y_test:{test_label.shape}')

x_train:(60000, 784)
y_train:(60000, 10)
x_test:(10000, 784)
y_test:(10000, 10)


In [12]:
weights = {"hidden_layer":tf.Variable(tf.compat.v1.random_normal([n_inputs, n_hidden_layer])), 
           "output":tf.Variable(tf.compat.v1.random_normal([n_hidden_layer, n_classes]))}

biases = {"hidden_layer": tf.Variable(tf.compat.v1.random_normal([n_hidden_layer])), 
          "output":tf.Variable(tf.compat.v1.random_normal([n_classes]))}

In [13]:
x = tf.compat.v1.placeholder(tf.float32, [None, n_inputs])
y = tf.compat.v1.placeholder(tf.float32, [None, n_classes])
print(x.shape)

(None, 784)


In [14]:
layer_1 = tf.add(tf.matmul(x, weights["hidden_layer"]), biases["hidden_layer"])
layer_1 = tf.nn.relu(layer_1)
logits = tf.add(tf.matmul(layer_1, weights["output"]), biases["output"])

In [20]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_train))
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=learn_rate).minimize(cost)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [22]:
init = tf.compat.v1.global_variables_initializer()

with tf.compat.v1.Session() as sess:
    sess.run(init)
    for i_ep in range(training_epochs):
        # for step, (batch_x, batch_y) in enumerate (train_data.take(display_step), 1):
        sess.run(optimizer, feed_dict={x:train_feature, y:train_label})
    test_accuracy = sess.run(accuracy, feed_dict={x:test_feature, y:test_label})

In [23]:
print(test_accuracy)

0.1036
